# 개체명 인식(Named Entity Recognition, NER) 모델 학습에 대한 기록

이 노트는 한국어 텍스트 데이터에서 특정 정보를 추출(예: 질의회신, 법령 내용)하기 위한 개체명 인식(NER) 딥러닝 모델을 학습하는 과정에 대한 기록입니다.

**학습 목표:**

1. Google Colab 환경 설정 및 Google Drive 연동.
2. 원본 텍스트 데이터 전처리 (Doccano 임포트 전).
3. Doccano를 이용한 웹 기반 데이터 라벨링.
4. 라벨링된 데이터를 딥러닝 모델 학습에 적합한 형태로 변환 (Hugging Face `datasets` 라이브러리).
5. 사전 학습된 BERT 모델(`klue/bert-base`)을 활용하여 NER 모델 파인튜닝.
6. 학습된 모델의 성능 평가 및 추론(Inference) 방법 이해.

**참고:** 이 튜토리얼은 Doccano와 Hugging Face `transformers` 라이브러리를 사용합니다.

---


---

### 0. 프로젝트 개요 및 동기

이 프로젝트는 회사에서 할당받은 업무를 효율적으로 해결하기 위해 시작되었습니다. 저에게 주어진 업무는 **각 공공기관 중앙부처의 법령해석 데이터셋을 웹 크롤링하여 PDF 또는 HWP 파일 내의 질의와 회신을 한 세트로 파싱하고 검증**하는 것이었습니다.

구체적으로는 다음과 같은 과제를 안고 있었습니다:

1.  **질의-회신 세트 개수 세기**: 웹 크롤링된 문서에서 '질의 1개'와 '회신 1개'가 정상적으로 하나의 세트를 이루는지 확인하고, 전체 세트의 개수를 파악해야 했습니다.
2.  **질의-회신 연결성 판단**: 질의와 회신 간의 연결이 논리적으로 올바른지, 즉 '질의 1'에 대한 '회신 1'이 제대로 매칭되는지 판단해야 했습니다.
3.  **특이사항 분류**: 특히 질의와 회신의 개수가 맞지 않거나, 회신 내용이 법령에 근거하지 않은 답변일 경우 이를 '특이사항'으로 분류하는 업무가 수반되었습니다.

문제는 이러한 분류 및 검증 작업을 수행해야 할 질의회신 세트가 **총 2만 개**에 달한다는 점이었습니다. 이는 엄청난 **수작업과 시간 소모**를 요구하는 일이었습니다. 컴퓨터 공학 전공자로서 이러한 비효율적인 업무를 제 전공 지식을 활용하여 자동화하고 해결하고자 하는 강한 동기를 느꼈습니다. 단순히 반복 작업을 줄이는 것을 넘어, 이 프로젝트를 통해 **딥러닝, 데이터 라벨링, 도커(Docker), 그리고 자연어 처리(NLP) 개념**을 실질적으로 배우고 적용하는 것을 목표로 삼았습니다. 이 보고서는 이러한 프로젝트에 대한 기록입니다.


### 프로젝트 전체 플로우 (이미지)

아래는 이 튜토리얼에서 진행할 전체 프로젝트의 워크플로우를 이미지로 시각화한 것입니다. (w/ mermaid)
![프로젝트 플로우차트](https://github.com/jaehunshin-git/deep-learning/blob/main/deep-learning-flowchart.png?raw=1)


## 프로젝트 전체 플로우 (텍스트 다이어그램)

아래는 이 튜토리얼에서 진행할 전체 프로젝트의 워크플로우를 텍스트 형태로 시각화한 것입니다. 각 단계의 흐름을 이해하는 데 도움이 될 것입니다.

```text
## 프로젝트 전체 플로우

1.  시작 - 딥러닝 NER 모델 학습
    -> 2. Google Colab 환경 설정
        -> 2.1 런타임 유형 GPU 변경
        -> 2.2 Google Drive 마운트
        -> 2.3 프로젝트 디렉토리 구조 설정
        -> 2.4 필요 라이브러리 설치

2.4 필요 라이브러리 설치
    -> 3. 데이터 준비 - 원본 텍스트 전처리 (로컬 PC)
        -> 3.1 원본 dataset.txt 로드
        -> 3.2 특정 문자/숫자/워딩 제거 - 정규표현식
        -> 3.3 각 질의/회신 쌍 분리 - 개행 문자 추가
        -> 3.4 dataset_cleaned_final.txt 저장

3.4 dataset_cleaned_final.txt 저장
    -> 4. Doccano 데이터 라벨링 (로컬 PC Docker)
        -> 4.1 Doccano Docker 설치 및 실행
        -> 4.2 Doccano 프로젝트 생성 - 레이블 정의
        -> 4.3 dataset_cleaned_final.txt Import - Plain Text
        -> 4.4 수동 개체명 라벨링 수행
        -> 4.5 라벨링된 데이터 Export - JSONL - Approved Only

4.5 라벨링된 데이터 Export - JSONL - Approved Only
    -> 5. Google Drive 업로드
        -> 6. Colab에서 라벨링된 데이터 로드 및 전처리
            -> 6.1 Doccano JSONL 파일 로드
            -> 6.2 BIO 태깅을 위한 레이블 목록 생성
            -> 6.3 텍스트 토큰화 - 레이블 정렬 - padding, truncation 포함
            -> 6.4 Hugging Face Dataset 객체로 변환
            -> 6.5 Train/Validation 데이터셋 분할

6.5 Train/Validation 데이터셋 분할
    -> 7. BERT 기반 NER 모델 학습
        -> 7.1 klue/bert-base 모델 및 토크나이저 로드
        -> 7.2 성능 지표 Metrics 정의 - seqeval
        -> 7.3 학습 인자 Training Arguments 설정 - eval_strategy 등
        -> 7.4 Trainer 객체 생성
        -> 7.5 trainer.train() 실행

7.5 trainer.train() 실행
    -> 8. 학습된 모델 평가 및 추론
        -> 8.1 학습된 모델 저장
        -> 8.2 저장된 모델 및 토크나이저 로드
        -> 8.3 단일 텍스트 추론 함수 정의
        -> 8.4 예시 문장으로 모델 테스트

8.4 예시 문장으로 모델 테스트
    -> 9. 결과 분석 - 모델 성능 평가
        -> 10. 성능이 충분한가?
            -> Yes -> 11. 배포 및 활용
            -> No  -> 12. 성능 향상 전략 적용
                -> 12.1 더 많은 데이터 라벨링 - 가장 중요!
                -> 12.2 하이퍼파라미터 튜닝
                -> 12.3 모델 아키텍처/전이 학습 고려
                -> 12.1 더 많은 데이터 라벨링 - 가장 중요! -> 4. Doccano 데이터 라벨링 (로컬 PC Docker)
```

---


## 0. Google Colab 환경 설정

모델 학습을 위해 Google Colab 환경을 설정합니다.

### 0.1 런타임 유형 변경 (GPU 활성화)

Colab에서 딥러닝 모델을 빠르게 학습시키려면 GPU를 사용하는 것이 필수적입니다.

1.  Colab 상단 메뉴에서 `런타임` (Runtime)을 클릭합니다.
2.  `런타임 유형 변경` (Change runtime type)을 선택합니다.
3.  `하드웨어 가속기` (Hardware accelerator) 드롭다운 메뉴에서 `GPU` 혹은 `TPU` 를 선택한 후 `저장` (Save)을 클릭합니다. - (`TPU` 선택 시 역번역 시간 단축 가능)

**주의:**

1.  런타임을 변경하거나 Colab 세션이 끊겼다가 다시 연결되면, 이전에 설치했던 라이브러리나 정의했던 변수들이 초기화됩니다. 이 경우 다음 단계를 다시 실행해야 합니다.
2.  google.colab 라이브러리는 colab 전용 라이브러리이므로 로컬에서는 설치 및 사용할 수 없습니다.


In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


### 0.3 프로젝트 디렉토리 구조 설정

Google Drive 내부에 프로젝트를 위한 폴더 구조를 설정합니다. 데이터와 모델을 체계적으로 관리할 수 있습니다.


In [2]:
import os

# 프로젝트의 루트 경로 설정 (이전에 당신이 설정한 Google Drive 경로)
# Colab Notebooks 폴더 아래에 deep-learning-ner-advanced 이라는 폴더를 만들어 이 프로젝트의 루트로 사용한다고 가정합니다.
# 이 경로는 사용자의 Google Drive 구조에 맞게 수정해주세요.

# 프로젝트 경로 설정
project_root = "/content/gdrive/MyDrive/Colab Notebooks/deep-learning-ner-advanced/"
data_dir = os.path.join(project_root, "data")
model_dir = os.path.join(project_root, "model")

os.makedirs(data_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

print(f"프로젝트 루트: {project_root}")
print(f"데이터 폴더: {data_dir}")
print(f"모델 저장 폴더: {model_dir}")

프로젝트 루트: /content/gdrive/MyDrive/Colab Notebooks/deep-learning-ner-advanced/
데이터 폴더: /content/gdrive/MyDrive/Colab Notebooks/deep-learning-ner-advanced/data
모델 저장 폴더: /content/gdrive/MyDrive/Colab Notebooks/deep-learning-ner-advanced/model


### 0.4 필요한 라이브러리 설치

BERT 모델을 사용하기 위한 Hugging Face `transformers`와 `datasets` 라이브러리를 설치합니다. `accelerate`는 학습 가속화에 도움을 줍니다.

1. `transformers`

   - 무엇인가요?

     - Hugging Face에서 개발한 가장 인기 있는 딥러닝 라이브러리 중 하나입니다. BERT, GPT-2, RoBERTa 등 다양한 사전 학습된 트랜스포머 기반 모델(Transformer-based models)과 토크나이저(Tokenizer)를 제공합니다. 자연어 처리(NLP) 분야의 최신 모델들을 쉽게 불러와 파인튜닝(fine-tuning)하거나 추론(inference)할 수 있도록 돕습니다.

   - 내 프로젝트에서 활용:

     - 모델 로드: `AutoModelForTokenClassification.from_pretrained("klue/bert-base")`를 사용하여 한국어에 특화된 BERT 모델인 klue/bert-base를 개체명 인식(Token Classification) 작업에 맞게 불러옵니다.

     - 토크나이저 로드: AutoTokenizer.from_pretrained("klue/bert-base")를 통해 텍스트를 모델이 이해할 수 있는 토큰(단어 조각) ID로 변환하는 데 필요한 토크나이저를 로드합니다.

     - 모델 학습 관리: Trainer 클래스를 사용하여 학습 인자(TrainingArguments) 설정, 학습 데이터와 평가 데이터 지정, 모델 학습(trainer.train()) 등 복잡한 학습 과정을 편리하게 관리합니다.

     - 모델 저장 및 로드: 학습된 모델을 저장하고 나중에 다시 불러와 추론에 활용하는 기능도 이 라이브러리를 통해 수행됩니다.

2. `datasets`

   - 무엇인가요?

     - Hugging Face에서 개발한 또 다른 핵심 라이브러리로, 대규모 데이터셋을 효율적으로 로드, 전처리, 저장할 수 있도록 설계되었습니다. 특히 NLP 작업에 최적화되어 있으며, 다양한 공개 데이터셋을 쉽게 가져올 수 있는 기능도 제공합니다.

   - 내 프로젝트에서 활용:

     - 데이터 로드: Doccano에서 내보낸 .jsonl 형식의 라벨링된 데이터를 로드하고 Pandas DataFrame을 거쳐 datasets.Dataset 객체로 변환합니다. Dataset 객체는 Hugging Face Trainer에 직접 입력될 수 있는 형태로, 데이터 처리를 효율적으로 만듭니다.

     - 데이터 전처리: Dataset 객체에 토크나이징된 input_ids, attention_mask, 그리고 각 토큰에 해당하는 labels (BIO 태그)를 저장하는 데 사용됩니다.

     - 데이터 분할: train_test_split 기능을 사용하여 라벨링된 전체 데이터를 학습용(train_dataset)과 평가용(eval_dataset)으로 쉽게 나눕니다.

3. `accelerate`

   - 무엇인가요?

     - Hugging Face에서 개발한 경량 라이브러리로, PyTorch 모델의 학습 과정을 분산 및 혼합 정밀도(mixed-precision) 학습 환경에 맞게 자동으로 조정해 줍니다. 개발자가 복잡한 분산 학습 코드를 직접 작성할 필요 없이, 소수의 코드 변경만으로 GPU 여러 개나 TPU 같은 가속기를 활용할 수 있도록 돕습니다.

   - 내 프로젝트에서 활용:

     - 당신의 Colab 환경에서 GPU 런타임을 사용하도록 설정했습니다. accelerate 라이브러리는 transformers의 Trainer 내부적으로 활용되어 GPU 자원을 효율적으로 사용하여 모델 학습 속도를 가속화합니다. 이는 대규모 모델과 데이터셋을 학습할 때 필수적인 역할을 합니다.

4. `seqeval`

   - 무엇인가요?

     - 시퀀스 라벨링(Sequence Labeling) 작업(개체명 인식, 품사 태깅 등)의 성능을 평가하기 위한 Python 라이브러리입니다. 특히 precision, recall, f1-score, accuracy와 같은 주요 지표들을 계산하는 데 특화되어 있습니다. 개체명 인식에서는 단순히 개별 토큰의 정확도뿐만 아니라, **전체 개체명 스팬(span)**이 정확하게 예측되었는지를 평가하는 것이 중요하며, seqeval이 이러한 역할을 수행합니다.

   - 내 프로젝트에서 활용:

     - `load_metric("seqeval")`를 사용하여 평가 지표를 로드합니다.

     - `compute_metrics` 함수 내에서 모델의 예측 결과와 실제 레이블을 seqeval 포맷에 맞게 변환하고, 이를 metric.compute() 함수에 전달하여 최종 정밀도, 재현율, F1-점수 등을 계산하는 데 사용됩니다. 이 지표들은 모델의 학습 과정을 모니터링하고 최종 성능을 평가하는 데 핵심적인 역할을 합니다.

이 라이브러리들은 모두 Hugging Face 생태계의 일부로, 복잡한 딥러닝 NLP 프로젝트를 효율적으로 개발하고 관리하는 데 큰 도움을 줍니다.


In [3]:
!pip install transformers[torch] evaluate datasets accelerate seqeval -q

!pip install --upgrade transformers

import numpy as np
import torch
import pandas as pd
from google.colab import drive
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, AutoModelForSeq2SeqLM
import evaluate
from tqdm.auto import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.6 MB/s eta 0:00:00


---


## 1. 데이터 준비 및 전처리 (Doccano 연동 전)

원본 텍스트 파일(`dataset.txt`)을 Doccano에 효율적으로 라벨링하기 위해 사전 전처리 작업을 수행합니다. 이 과정은 불필요한 노이즈를 제거하고, 각 질의/회신 쌍을 Doccano가 개별 문서로 인식할 수 있도록 분리하는 역할을 합니다.

**원본 `dataset.txt` 파일의 문제점:**

- 불필요한 숫자나 특정 워딩(예: '2021년 소방시설법령 질의회신집') 포함.
- 모든 질의/회신이 하나의 긴 텍스트로 이어져 있어, Doccano에서 개별 문서로 처리하기 어려움.

이 전처리 스크립트는 **Google Colab이 아닌, 로컬 환경(예: VS Code)**에서 원본 텍스트 파일을 처리한 후, 그 결과 파일을 Google Drive에 업로드하여 Doccano로 가져오는 것을 권장합니다.


### 1.1 `dataset.txt` 파일 클리닝 및 문서 분리 코드

아래 코드를 로컬 컴퓨터의 `.py` 파일로 저장(예: `preprocess_dataset.py`)하고, 원본 `dataset.txt` 파일이 있는 동일한 폴더에서 실행하세요.

```python
import os
import re

def clean_and_prepare_text_for_doccano_final_v2(input_filepath, output_filepath):
    """
    주어진 텍스트 파일에서 다음을 수행합니다:
    1. 한 줄에 숫자(0-9) 하나만 있는 라인을 삭제합니다.
    2. '2021년 소방시설법령 질의회신집' 문자열을 제거합니다.
    3. '질의 N.' (또는 '질의 N')으로 시작하는 줄 앞에 빈 줄(\n\n)을 추가합니다.
       (단, 파일의 맨 처음 나오는 '질의 1.' 앞에는 추가하지 않습니다.)
       이때, 각 '질의 N.' 블록이 정확히 '\n\n'으로 구분되도록 합니다.
    """
    try:
        with open(input_filepath, 'r', encoding='utf-8') as f_in:
            lines = f_in.readlines() # 파일을 줄 단위로 읽어옵니다.

        temp_content = [] # 임시로 클리닝된 줄을 저장할 리스트
        for line in lines:
            # 1단계: '2021년 소방시설법령 질의회신집' 문자열 제거
            line = line.replace('2021년 소방시설법령 질의회신집', '')

            # 2단계: 한 줄에 숫자 하나만 있는 라인 삭제
            if line.strip().isdigit() and len(line.strip()) == 1:
                continue # 해당 줄은 건너뛰고 다음 줄로 넘어갑니다.

            # 모든 줄의 양쪽 공백 제거 후 임시 리스트에 추가 (원래 줄바꿈도 제거)
            temp_content.append(line.strip())

        # 임시 리스트의 줄들을 하나의 문자열로 결합 (각 줄 사이에 공백 1개로 연결)
        cleaned_raw_text = ' '.join(temp_content).strip()

        # 3단계: '질의 N.' 앞에 빈 줄 추가
        # '질의 N.' (또는 '질의 N') 패턴을 찾아서 '\n\n질의 N.'으로 교체합니다.
        # 단, 파일의 맨 처음 나오는 '질의 1.' 앞에는 추가하지 않습니다.

        # re.sub의 repl 매개변수에 함수를 사용하여 동적 교체
        def replace_query_marker(match):
            # match.start() == 0 이면 파일의 맨 처음 '질의 N.'입니다.
            if match.start() == 0:
                return match.group(0) # '질의 N.' 자체를 반환 (앞에 아무것도 안 붙임)
            else:
                return '\n\n' + match.group(0) # 그 외의 '질의 N.' 앞에는 '\n\n'을 붙임

        # 패턴: '질의' 다음에 공백(0개 이상), 숫자(1개 이상), 점(선택적)
        final_content = re.sub(r'질의\s*\d+\.?', replace_query_marker, cleaned_raw_text)

        final_content = re.sub(r'(회신\s*\d\.?)', r'\n\1', final_content)

        final_content = re.sub(r'▣.*', '', final_content)  # '▣'로 시작하는 줄 제거

        final_content = re.sub(r'\s\d{2,3}\s', '', final_content)  # 숫자(2-3자리) 제거

        # 최종적으로 문자열의 시작과 끝에 불필요한 공백/개행을 제거
        final_content = final_content.strip()

        with open(output_filepath, 'w', encoding='utf-8') as f_out:
            f_out.write(final_content)

        print(f"파일이 성공적으로 처리되어 '{output_filepath}'에 저장되었습니다.")
        print("이 파일을 Doccano에 'Plain Text' 형식으로 가져오시면 됩니다.")

    except FileNotFoundError:
        print(f"오류: 입력 파일 '{input_filepath}'을(를) 찾을 수 없습니다. 경로를 확인하세요.")
    except Exception as e:
        print(f"파일 처리 중 오류 발생: {e}")

# --- 사용 방법 (아래 경로를 당신의 실제 파일 경로로 수정해주세요) ---

# 로컬 프로젝트 루트 경로 (VS Code에서 해당 파일이 있는 폴더 경로)
# 예: r"C:\Users\JHSHIN\ProgrammingCodes\deep-learning"
project_root = r"C:\Users\JHSHIN\ProgrammingCodes\deep-learning"

# 원본 입력 파일 경로 (당신이 가지고 있는 원본 dataset.txt 파일)
input_file = os.path.join(project_root, 'dataset.txt')

# 수정된 내용을 저장할 새로운 출력 파일 경로 (새로운 이름으로 저장하는 것을 권장)
output_file = os.path.join(project_root, 'dataset_cleaned.txt') # 파일 이름 변경

# 함수 실행
clean_and_prepare_text_for_doccano_final_v2(input_file, output_file)
```


**원본 `dataset.txt` 파일 예시:**

질의 1
연면적 450㎡인 특정소방대상물에 최초 건축물 사용승인시에 비상경보설비 설치가 되지
않은 경우 건축허가일과 사용승인일 중 소방시설설치기준 적용일은?
회신 1
건축물 등의 신축ㆍ증축ㆍ개축ㆍ재축ㆍ이전ㆍ용도변경 또는 대수선의 허가
ㆍ협의 및 사용승인의 권한이 있는 행정기관은 소방시설법 제7조제1항에
따라 소재지를 관할하는 소방본부장이나 소방서장의 동의를 받아야 하므로,
건축허가등과 관련한 협의과정이 누락되었다면, 건축허가 신청일을 기준으로
소방시설의 설치기준을 적용합니다.
질의 2
최초 사업허가승인월이 ‘13년 6월인 대상물의 사업이 변경되어 최종 사업허가승인월이
19년 2월인 경우, 소방시설법 적용 기준일은?
회신 2
소방시설설치기준 적용 기준일은 최초 사용승인계획 신청 시점입니다.2021년 소방시설법령 질의회신집
4
최초 건축허가과정에서 허가동의된 사업계획은 이후 사업 변경계획이 신청
되어도 변경계획이 신청된 시점의 소방시설법을 적용하지 않습니다. 부칙
<대통령령 제27810호, 2017.1.26.>호제2조 소방시설 설치에 관한 적용례에
관한 적용례에 특정소방대상물의 신축ㆍ증축ㆍ개축ㆍ재축ㆍ이전ㆍ용도변경
ㆍ대수선의 허가ㆍ협의를 신청하거나 신고하는 경우로 명시하고 있어, 사용
승인계획변경 등 허가의 변경사항은 개정 규정 적용대상에 해당하지 않습
니다.
▣ 건축허가등의 동의대상 범위
[소방시설법 시행령 제12조]
관계법령
제12조(건축허가등 동의대상물의 범위 등)
① 법 제7조제1항에 따라 건축허가등을 할 때 미리 소방본부장 또는 소방서장의 동의를
받아야 하는 건축물 등의 범위는 다음 각 호와 같다.
1. 연면적(「건축법 시행령」 제119조제1항제4호에 따라 산정된 면적을 말한다. 이하 같
다)이 400제곱미터 이상인 건축물. 다만, 다음 각 목의 어느 하나에 해당하는 시설은
해당 목에서 정한 기준 이상인 건축물로 한다.

**`preprocessing_dataset.py` 실행 후 `dataset_cleaned.txt` 예시:**


질의 1 연면적 450㎡인 특정소방대상물에 최초 건축물 사용승인시에 비상경보설비 설치가 되지 않은 경우 건축허가일과 사용승인일 중 소방시설설치기준 적용일은? 회신 1 건축물 등의 신축ㆍ증축ㆍ개축ㆍ재축ㆍ이전ㆍ용도변경 또는 대수선의 허가 ㆍ협의 및 사용승인의 권한이 있는 행정기관은 소방시설법 제7조제1항에 따라 소재지를 관할하는 소방본부장이나 소방서장의 동의를 받아야 하므로, 건축허가등과 관련한 협의과정이 누락되었다면, 건축허가 신청일을 기준으로 소방시설의 설치기준을 적용합니다.

질의 2 최초 사업허가승인월이 ‘13년 6월인 대상물의 사업이 변경되어 최종 사업허가승인월이 19년 2월인 경우, 소방시설법 적용 기준일은? 회신 2 소방시설설치기준 적용 기준일은 최초 사용승인계획 신청 시점입니다. 최초 건축허가과정에서 허가동의된 사업계획은 이후 사업 변경계획이 신청 되어도 변경계획이 신청된 시점의 소방시설법을 적용하지 않습니다. 부칙 <대통령령 제27810호, 2017.1.26.>호제2조 소방시설 설치에 관한 적용례에 관한 적용례에 특정소방대상물의 신축ㆍ증축ㆍ개축ㆍ재축ㆍ이전ㆍ용도변경 ㆍ대수선의 허가ㆍ협의를 신청하거나 신고하는 경우로 명시하고 있어, 사용 승인계획변경 등 허가의 변경사항은 개정 규정 적용대상에 해당하지 않습 니다.


---


## 2. Doccano를 이용한 데이터 라벨링

Doccano는 텍스트 어노테이션(라벨링)을 위한 오픈소스 도구입니다. 웹 기반 환경에서 직관적으로 개체명(NER) 라벨링을 수행할 수 있습니다.

---


### **(New) 2.0 약 지도 학습 (Weak Supervision)으로 라벨링 가속화** 🚀

수천 개의 데이터를 처음부터 수동으로 라벨링하는 것은 매우 힘든 작업입니다. **약 지도 학습**은 정규 표현식(Regex)과 같은 간단한 **규칙(Heuristics)을 이용해 대량의 데이터에 자동으로 라벨을 부여**하는 기법입니다.

- 정규표현식(Regex)과 같은 간단한 규칙을 사용하여 '질의 1', '회신 1' 등 명확한 패턴을 가진 개체를 자동으로 라벨링합니다.

- 이렇게 생성된 "초벌 라벨링" 데이터를 Doccano에 임포트하여 검토 및 수정만 하면 되므로, 라벨링 시간을 획기적으로 단축할 수 있습니다.


In [ ]:
import re
import json
import os

def create_weak_labels_advanced(input_text_path, output_jsonl_path):
    """
    고도로 상세화된 정규식 휴리스틱을 사용해 자동으로 라벨을 생성합니다.
    - ID, QUESTION_CONTENT, ANSWER_CONTENT는 구조 기반으로 라벨링
    - LAW_CONTENT는 상세화된 복합 패턴을 적용하여 정교하게 라벨링
    """
    try:
        with open(input_text_path, 'r', encoding='utf-8') as f:
            documents = f.read().strip().split('\n\n')
    except FileNotFoundError:
        print(f"오류: '{input_text_path}' 파일을 찾을 수 없습니다. 경로를 확인하세요.")
        return

    # --- 라벨링 규칙(패턴) 상세화 ---
    # ID 패턴
    question_id_pattern = re.compile(r'질의\s*\d+\.?(?=\s|$)')
    answer_id_pattern = re.compile(r'회신\s*\d+\.?(?=\s|$)')

    # LAW_CONTENT를 찾기 위한 고도로 상세화된 휴리스틱(Heuristics) 패턴
    # 우선순위가 높은(더 구체적인) 패턴을 리스트의 위쪽에 배치합니다.
    law_patterns = [
        # 1. 「...법」, 『...기준』 등 특수기호로 감싸진 법률/기준 이름 (가장 강력한 패턴)
        re.compile(r'(?:「|『)[^」』]+(?:법|법률|시행령|시행규칙|기준)(?:」|』)'),

        # 2. '소방시설법 시행령 제12조제1항제1호' 와 같이 법 이름과 조항이 함께 나오는 패턴
        re.compile(r'[^」』\s]+(?:법|령|규칙|기준)\s?제\s?\d+조(?:\s?제\s?\d+항)?(?:\s?제\s?\d+호)?'),

        # 3. '[소방시설법 제9조]' 와 같이 대괄호로 감싸진 법률 및 조항
        re.compile(r'\[\s?[^\]]+(?:법|령|기준|조)\s?[^\]]*\]'),

        # 4. '[별표5]' 와 같이 별표/서식을 나타내는 패턴
        re.compile(r'\[\s?별표\s?\d+\s?\]'),

        # 5. '제N조 제N항 제N호' 등 법률 조항만 단독으로 나오는 패턴
        re.compile(r'제\s?\d+조(?:\s?제\s?\d+항)?(?:\s?제\s?\d+호)?'),

        # 6. '화재안전기준' 등 단독으로 쓰이는 핵심 법규/기준 키워드
        re.compile(r'화재안전기준'),

        # 7. '...법에 따라' 등 법적 근거를 제시하는 표현 (가장 범위가 넓으므로 마지막에 배치)
        re.compile(r'\S+법[에\s](?:따라|따르면|의하면|근거하여)')
    ]

    all_labeled_docs = []
    print(f"총 {len(documents)}개의 문서에 대해 향상된 약 지도 학습을 시작합니다...")

    for doc_text in documents:
        if not doc_text.strip():
            continue

        spans = []
        markers = []

        # 1. 문서 내 모든 ID 마커의 위치와 종류를 찾음
        for match in question_id_pattern.finditer(doc_text):
            markers.append({'start': match.start(), 'end': match.end(), 'type': 'QUESTION_ID'})
        for match in answer_id_pattern.finditer(doc_text):
            markers.append({'start': match.start(), 'end': match.end(), 'type': 'ANSWER_ID'})

        markers.sort(key=lambda x: x['start'])
        if not markers:
            continue

        # 2. 마커를 기준으로 잘라가며 CONTENT 라벨링
        for i in range(len(markers)):
            current_marker = markers[i]
            spans.append([current_marker['start'], current_marker['end'], current_marker['type']])

            content_start = current_marker['end']
            content_end = markers[i+1]['start'] if i + 1 < len(markers) else len(doc_text)

            content_text_segment = doc_text[content_start:content_end]
            lstrip_len = len(content_text_segment) - len(content_text_segment.lstrip())
            content_start += lstrip_len
            content_end -= (len(content_text_segment) - len(content_text_segment.rstrip()))

            if content_start >= content_end: continue

            if current_marker['type'] == 'QUESTION_ID':
                spans.append([content_start, content_end, 'QUESTION_CONTENT'])

            elif current_marker['type'] == 'ANSWER_ID':
                answer_block_text = doc_text[content_start:content_end]
                law_spans_in_block = []

                # 3. ANSWER_CONTENT 내에서 모든 LAW_CONTENT 패턴 찾기
                for pattern in law_patterns:
                    for match in pattern.finditer(answer_block_text):
                        law_start = content_start + match.start()
                        law_end = content_start + match.end()
                        law_spans_in_block.append([law_start, law_end, 'LAW_CONTENT'])

                if not law_spans_in_block:
                    spans.append([content_start, content_end, 'ANSWER_CONTENT'])
                    continue

                # 4. 찾은 LAW_CONTENT들을 병합하고, 그 외 부분을 ANSWER_CONTENT로 라벨링
                law_spans_in_block.sort(key=lambda x: x[0])

                # 중첩/겹치는 부분을 병합 (Merge overlapping spans)
                merged_law_spans = []
                if law_spans_in_block:
                    current_span = law_spans_in_block[0]
                    for next_span in law_spans_in_block[1:]:
                        if next_span[0] < current_span[1]: # 겹치는 경우
                            current_span[1] = max(current_span[1], next_span[1])
                        else:
                            merged_law_spans.append(current_span)
                            current_span = next_span
                    merged_law_spans.append(current_span)

                # LAW_CONTENT를 제외한 나머지 부분을 ANSWER_CONTENT로 채우기
                last_end = content_start
                for law_start, law_end, law_label in merged_law_spans:
                    if law_start > last_end:
                        spans.append([last_end, law_start, 'ANSWER_CONTENT'])
                    spans.append([law_start, law_end, law_label])
                    last_end = law_end

                if content_end > last_end:
                    spans.append([last_end, content_end, 'ANSWER_CONTENT'])

        spans.sort(key=lambda x: x[0])
        all_labeled_docs.append({"text": doc_text, "labels": spans})

    with open(output_jsonl_path, 'w', encoding='utf-8') as f:
        for doc in all_labeled_docs:
            f.write(json.dumps(doc, ensure_ascii=False) + '\n')

    print(f"고도로 상세화된 약 지도 학습 완료! {len(all_labeled_docs)}개의 문서가 '{output_jsonl_path}'에 저장되었습니다.")

# --- 함수 실행 ---
# 아래 경로들은 실제 환경에 맞게 설정해야 합니다.
# project_root = os.path.dirname(os.path.abspath(__file__)) # local 환경에서 실행 시 주석 해제 (현재 파일의 경로를 기준으로 설정)

print(f"프로젝트 루트 경로: {project_root}")

after_perprocessing = os.path.join(project_root, 'data', 'dataset_cleaned.txt')
weakly_labeled_path = os.path.join(project_root, 'data', 'weakly_labeled_advanced.jsonl')
# weakly_labeled_path = os.path.join('data_dir', 'weakly_labeled_advanced.json')

# # 파일이 존재할 때만 실행
if os.path.exists(after_perprocessing):
    create_weak_labels_advanced(after_perprocessing, weakly_labeled_path)
else:
    print(f"입력 파일 '{after_perprocessing}'를 찾을 수 없어 실행을 건너뜁니다.")

프로젝트 루트 경로: /content/gdrive/MyDrive/Colab Notebooks/deep-learning-ner-advanced/
총 309개의 문서에 대해 향상된 약 지도 학습을 시작합니다...
고도로 상세화된 약 지도 학습 완료! 302개의 문서가 '/content/gdrive/MyDrive/Colab Notebooks/deep-learning-ner-advanced/data/weakly_labeled_advanced.jsonl'에 저장되었습니다.


### 2.1 Doccano 설치 및 실행 (Docker 사용)

Doccano는 Docker를 이용하여 가장 쉽게 설치하고 실행할 수 있습니다. 로컬 컴퓨터에 Docker가 설치되어 있어야 합니다.

1.  **Docker 설치:** Docker Desktop을 다운로드하여 설치합니다: [https://www.docker.com/products/docker-desktop/](https://www.docker.com/products/docker-desktop/)

2.  **Doccano Docker 이미지 다운로드 및 실행:** 터미널 또는 명령 프롬프트에서 다음 명령어를 실행합니다.

    ```bash
    docker pull doccano/doccano
    docker run -it -p 8000:8000 doccano/doccano
    ```

3.  **Doccano 접속:** 웹 브라우저를 열고 `http://localhost:8000/`으로 접속합니다. 기본 관리자 계정은 `admin / admin`입니다.

### 2.2 Doccano 프로젝트 생성 및 설정

1.  **새 프로젝트 생성:** Doccano 웹 UI에서 `Create new project` 버튼을 클릭합니다.
2.  **프로젝트 이름 및 설명 입력:** 프로젝트 이름을 지정하고(예: '법령 질의회신 NER') 설명을 추가합니다.
3.  **프로젝트 유형 선택:** `Sequence Labeling` (개체명 인식을 위한 유형)을 선택합니다.
4.  **레이블 정의:** NER 모델이 인식할 개체명 레이블들을 정의합니다. 예를 들어:

    - `QUESTION_ID`
    - `QUESTION_CONTENT`
    - `ANSWER_ID`
    - `ANSWER_CONTENT`
    - `MISC_HEADER` (예: 고시 번호, 문서 제목)
    - `LAW_CONTENT` (예: 특정 법령 조항, 법률 이름)

    각 레이블에 단축키와 색상을 지정하면 라벨링 효율을 높일 수 있습니다.

### 2.3 클리닝된 데이터 Doccano로 가져오기 (Import)

1.  **Import Data 탭 이동:** 생성된 프로젝트 페이지에서 `Import Data` 탭을 클릭합니다.
2.  **파일 선택:** `dataset_cleaned_final.txt` 파일을 선택합니다.
3.  **파일 형식 선택:** `Plain Text`를 선택합니다. (우리가 `\n\n`으로 문서를 분리했기 때문에, Doccano가 이를 개별 문서로 인식합니다.)
4.  **Import 시작:** `Import` 버튼을 클릭합니다.

### 2.4 개체명 라벨링 수행

1.  **Annotate Data 탭 이동:** `Annotate Data` 탭을 클릭합니다.
2.  **텍스트 선택 및 라벨 지정:** 각 문서의 텍스트를 읽고, 해당하는 단어나 구절을 드래그하여 선택한 후, 오른쪽에 나타나는 레이블 버튼(또는 지정된 단축키)을 클릭하여 라벨을 지정합니다.
3.  **작업 저장 및 다음 문서로 이동:** 한 문서의 라벨링을 마쳤으면 `Submit` 또는 `Save` 버튼을 눌러 저장하고 다음 문서로 넘어갑니다.

### 2.5 라벨링 완료 후 데이터 내보내기 (Export)

충분한 양의 데이터를 라벨링했다면 (초기 학습을 위해 최소 100개 이상, 실사용을 위해 수백~수천 개 권장), 이제 라벨링된 데이터를 내보냅니다.

1.  **Export Data 탭 이동:** 프로젝트 페이지에서 `Export Data` 탭을 클릭합니다.
2.  **파일 형식 선택:** `JSONL`을 선택합니다. (Hugging Face `datasets` 라이브러리가 쉽게 로드할 수 있는 형식입니다.)
3.  **'Export only approved documents' 체크 (매우 중요!)**:
    이 옵션을 **반드시 체크**합니다. 이 옵션은 라벨링이 완료되고 `Approve` (승인)된 문서만 내보내어 학습 데이터의 품질을 보장합니다. 승인되지 않은 문서는 아직 검토가 필요하거나 수정될 여지가 있는 것으로 간주됩니다.

    - **Approve 하는 방법:** `Annotate` 화면에서 각 문서의 라벨링을 마친 후 `Approve` 버튼을 클릭하거나, `Annotation` -> `Guideline` -> `Approve` 로 이동하여 일괄 승인할 수 있습니다.

4.  **Export 시작:** `Export` 버튼을 클릭하여 `.jsonl` 파일을 다운로드합니다.

**다운로드한 `.jsonl` 파일을 Google Drive의 `data` 폴더(예: `/content/gdrive/MyDrive/Colab Notebooks/deep-learning/data/`)로 업로드합니다.** 파일 이름을 기억해두세요 (예: `after_datalabeling.jsonl`).


---


## 3. Colab에서 라벨링된 데이터 로드 및 전처리

Doccano에서 내보낸 `.jsonl` 파일을 Google Colab으로 가져와 BERT 모델 학습에 적합한 형태로 전처리합니다. 이 과정에서 각 단어(또는 서브워드 토큰)에 BIO(Beginning, Inside, Outside) 태그를 할당합니다.

**⚠️ 중요: Colab 런타임이 끊어졌다면, 반드시 `0.2 Google Drive 마운트`, `0.3 프로젝트 디렉토리 구조 설정`, `0.4 필요한 라이브러리 설치` 셀을 다시 실행한 후 이 단계를 진행해주세요.**


In [ ]:
import os
import json
import pandas as pd
from datasets import Dataset, Features, Value, ClassLabel, Sequence
from transformers import AutoTokenizer
from pprint import pprint

# --- 1. Doccano에서 정의한 레이블 목록 (B-, I- 없이) ---
# Doccano에서 프로젝트 설정 시 정의했던 레이블 이름들을 여기에 정확히 입력합니다.
doccano_raw_labels = [
    "QUESTION_ID",
    "QUESTION_CONTENT",
    "ANSWER_ID",
    "ANSWER_CONTENT",
    # "MISC_HEADER",
    "LAW_CONTENT",
]

# --- 2. 모델 학습을 위한 최종 BIO 레이블 목록 및 매핑 ---
# 'O' (Other) 태그는 라벨링되지 않은 모든 토큰을 의미하며 항상 포함됩니다.
# 각 원본 레이블에 대해 'B-' (Beginning)와 'I-' (Inside) 태그를 생성합니다.
label_list = ["O"]  # 'Other' 태그는 항상 포함
for label in doccano_raw_labels:
    label_list.append(f"B-{label}")  # Beginning 태그: 개체명의 첫 번째 토큰
    label_list.append(f"I-{label}")  # Inside 태그: 개체명의 두 번째 이후 토큰

# 레이블 이름과 정수 ID 간의 매핑을 생성합니다.
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}
num_labels = len(label_list)  # 모델의 출력 레이어 크기에 사용됩니다.

print(f"모델 학습을 위한 최종 레이블 목록: {label_list}")
print(f"총 레이블 개수: {num_labels}")

pprint(label_to_id)
pprint(id_to_label)


# --- 3. Doccano에서 내보낸 JSONL 파일 경로 설정 ---
# 이 경로는 0.3단계에서 설정한 data_dir과 일치해야 합니다.
# Colab 런타임이 재시작되면 변수가 초기화될 수 있으므로, 방어 코드를 추가하거나 0.3단계 셀을 다시 실행해야 합니다.
try:
    # project_root가 정의되지 않았다면 (런타임 재시작 등), 기본 경로를 설정
    if "project_root" not in locals():
        project_root = "/content/gdrive/MyDrive/Colab Notebooks/deep-learning/"
        data_dir = os.path.join(project_root, "data")
        print("경로 변수 'project_root'가 정의되지 않아 기본 경로를 사용합니다.")
except NameError:
    # NameError 발생 시 (아예 변수 선언이 안 되어 있을 때)
    project_root = "/content/gdrive/MyDrive/Colab Notebooks/deep-learning/"
    data_dir = os.path.join(project_root, "data")
    print("경로 변수 'project_root'가 정의되지 않아 기본 경로를 설정했습니다.")

# Doccano에서 내보낸 실제 JSONL 파일 이름을 여기에 입력하세요.
labeled_data_file_path = os.path.join(data_dir, "after_datalabeling.jsonl")

# --- 4. 사용할 토크나이저 ---
# 한국어 BERT 모델인 'klue/bert-base' 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

# --- 5. Doccano JSONL 로드 및 Hugging Face Dataset 형식으로 변환 ---
converted_data_for_hf = []
try:
    with open(labeled_data_file_path, "r", encoding="utf-8") as f:
        for line in f:
            doc_data = json.loads(line)
            pprint(doc_data)
            text = doc_data["text"]
            pprint(text)
            # Doccano의 'label' 필드는 [[start_offset, end_offset, "LABEL_NAME"], ...] 형식입니다.
            annotations = doc_data.get("labels", [])
            pprint(annotations)
            # 텍스트를 토큰화하고 각 토큰의 원본 텍스트에서의 위치(offset)를 함께 가져옵니다.
            # 중요: padding과 truncation을 활성화하여 모든 시퀀스 길이를 통일합니다.
            tokenized_output = tokenizer(
                text,
                return_offsets_mapping=True,  # 토큰의 원본 텍스트에서의 시작/끝 오프셋 반환
                truncation=True,  # max_length를 초과하는 시퀀스는 잘라냄
                max_length=512,  # BERT 모델의 최대 입력 길이 (일반적으로 512)
                padding="max_length",  # 모든 시퀀스를 max_length에 맞춰 패딩
            )

            input_ids = tokenized_output["input_ids"]
            offsets = tokenized_output["offset_mapping"]

            # 각 토큰에 해당하는 BIO 레이블을 초기화합니다.
            # -100은 손실 계산에서 무시될 특수 토큰(CLS, SEP 등)이나 패딩 토큰에 할당됩니다.
            labels = [-100] * len(input_ids)

            word_ids = tokenized_output.word_ids(
                batch_index=0
            )  # 토큰이 어떤 원본 단어에 해당하는지 ID 매핑

            # 토큰별로 레이블 할당 (Doccano의 스팬 기반 레이블을 토큰 기반 BIO 레이블로 변환)
            for token_idx, word_idx in enumerate(word_ids):
                if word_idx is None:  # CLS, SEP, 패딩 토큰과 같은 특수 토큰
                    labels[token_idx] = -100  # 손실 계산에서 무시
                else:  # 일반 단어에 해당하는 토큰
                    # 현재 토큰의 원본 텍스트에서의 시작/끝 오프셋
                    token_start_offset = offsets[token_idx][0]
                    token_end_offset = offsets[token_idx][1]

                    current_token_label_name = (
                        "O"  # 현재 토큰의 기본 레이블은 "O" (Other)
                    )

                    # 현재 토큰이 어떤 어노테이션에 속하는지 확인합니다.
                    for ann_start, ann_end, ann_label_name in annotations:
                        # 토큰의 오프셋이 어노테이션 범위 내에 완전히 포함되는 경우
                        if (
                            ann_start <= token_start_offset
                            and token_end_offset <= ann_end
                        ):
                            # 만약 현재 토큰의 시작 오프셋이 어노테이션의 시작 오프셋과 같다면 B- 태그
                            if ann_start == token_start_offset:
                                current_token_label_name = f"B-{ann_label_name}"
                            # 아니라면 I- 태그
                            else:
                                current_token_label_name = f"I-{ann_label_name}"
                            break  # 해당 어노테이션을 찾았으니 더 이상 검색할 필요 없음

                    labels[token_idx] = label_to_id[current_token_label_name]

            # 실제 모델 입력에 필요한 형태로 데이터를 저장합니다.
            converted_data_for_hf.append(
                {
                    "input_ids": input_ids,
                    "attention_mask": tokenized_output["attention_mask"],
                    "labels": labels,  # 이 labels는 ID로 변환된 BIO 태그 리스트
                }
            )

    # Python 리스트를 Pandas DataFrame으로 변환 후 Hugging Face Dataset으로 변환
    processed_df = pd.DataFrame(converted_data_for_hf)
    # 특징(features)을 명시적으로 정의하여 Dataset이 올바른 데이터 타입을 갖도록 합니다.
    # 이는 특히 ClassLabel과 같은 특정 타입의 데이터에 중요합니다.
    features = Features(
        {
            "input_ids": Sequence(Value("int32")),
            "attention_mask": Sequence(Value("int32")),
            "labels": Sequence(
                ClassLabel(names=label_list)
            ),  # labels는 ClassLabel 시퀀스
        }
    )
    hf_dataset = Dataset.from_pandas(processed_df, features=features)

    print(f"\nHugging Face Dataset으로 변환된 샘플 수: {len(hf_dataset)}")
    print("\n변환된 Hugging Face Dataset 첫 번째 샘플:")
    print(hf_dataset[0])
    print(
        f"디코딩된 토큰: {tokenizer.convert_ids_to_tokens(hf_dataset[0]['input_ids'])}"
    )
    decoded_labels = [
        id_to_label[l_id] if l_id != -100 else "O" for l_id in hf_dataset[0]["labels"]
    ]
    print(f"디코딩된 레이블: {decoded_labels}")
    # 원본 텍스트 디코딩
    print(
        f"디코딩된 텍스트: {tokenizer.decode(hf_dataset[0]['input_ids'], skip_special_tokens=True)}"
    )

    # --- 6. 데이터셋 분할 (Train/Validation Split) ---
    # 모델 학습을 위해 전체 데이터셋을 학습(train) 세트와 평가(validation) 세트로 나눕니다.
    # test_size=0.2는 전체 데이터의 20%를 평가 세트로 사용한다는 의미입니다.
    # seed는 재현 가능한 결과를 위해 설정합니다.
    train_test_split_dataset = hf_dataset.train_test_split(test_size=0.2, seed=42)
    train_dataset = train_test_split_dataset["train"]
    eval_dataset = train_test_split_dataset["test"]

    print(f"\n학습 데이터셋 샘플 수: {len(train_dataset)}")
    print(f"평가 데이터셋 샘플 수: {len(eval_dataset)}")

except FileNotFoundError:
    print(
        f"오류: 레이블링된 파일 '{labeled_data_file_path}'을(를) 찾을 수 없습니다. Google Drive에 업로드했는지 확인하세요."
    )
except Exception as e:
    print(f"레이블링된 파일을 로드하거나 처리하는 중 오류 발생: {e}")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 [5, 168, 'QUESTION_CONTENT'],
 [169, 173, 'ANSWER_ID'],
 [174, 185, 'ANSWER_CONTENT'],
 [185, 193, 'LAW_CONTENT'],
 [193, 209, 'ANSWER_CONTENT'],
 [209, 212, 'LAW_CONTENT'],
 [212, 304, 'ANSWER_CONTENT'],
 [304, 312, 'LAW_CONTENT'],
 [312, 332, 'ANSWER_CONTENT'],
 [332, 364, 'LAW_CONTENT'],
 [364, 365, 'ANSWER_CONTENT'],
 [365, 368, 'LAW_CONTENT'],
 [368, 551, 'ANSWER_CONTENT']]
{'Comments': [],
 'id': 768,
 'label': [],
 'labels': [[0, 4, 'QUESTION_ID'],
            [5, 70, 'QUESTION_CONTENT'],
            [71, 75, 'ANSWER_ID'],
            [76, 150, 'ANSWER_CONTENT'],
            [150, 158, 'LAW_CONTENT'],
            [158, 161, 'ANSWER_CONTENT'],
            [161, 168, 'LAW_CONTENT'],
            [168, 170, 'ANSWER_CONTENT'],
            [170, 177, 'LAW_CONTENT'],
            [177, 209, 'ANSWER_CONTENT'],
            [209, 216, 'LAW_CONTENT'],
            [216, 269, 'ANSWER_CONTENT'],
            [269, 272, 'LAW_CONTENT'],
            [272, 459, 

---


## 4. BERT 기반 NER 모델 학습

이제 전처리된 데이터를 사용하여 `klue/bert-base` 모델을 개체명 인식 작업에 맞게 파인튜닝합니다. Hugging Face `Trainer` API를 사용하면 학습 과정을 매우 편리하게 관리할 수 있습니다.

**⚠️ 중요: Colab 런타임이 끊어졌다면, 반드시 `0.2 Google Drive 마운트`, `0.3 프로젝트 디렉토리 구조 설정`, `0.4 필요한 라이브러리 설치` 셀과 함께 `3. Colab에서 라벨링된 데이터 로드 및 전처리` 셀을 다시 실행한 후 이 단계를 진행해주세요.**


In [ ]:
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
import torch
import numpy as np

# 1. 모델 로드
# AutoModelForTokenClassification은 토큰 분류(NER과 같은 작업)를 위한 모델입니다.
# num_labels는 당신의 label_list에 있는 최종 레이블(BIO 태그 포함)의 개수입니다.
# id_to_label과 label2id는 모델이 숫자 ID와 레이블 이름을 매핑하는 데 사용됩니다.
model = AutoModelForTokenClassification.from_pretrained(
    "klue/bert-base",
    num_labels=num_labels,  # 우리의 최종 BIO 레이블 개수
    id2label=id_to_label,  # ID를 레이블 이름으로 매핑
    label2id=label_to_id,  # 레이블 이름을 ID로 매핑
)

print("모델과 토크나이저 로드 완료.")
print(
    """
주의: 'Some weights of BertForTokenClassification were not initialized...' 메시지는 정상입니다.
이는 사전 학습된 BERT 모델에 토큰 분류를 위한 새로운 레이어(classifier.bias, classifier.weight)가 추가되었기 때문입니다.
이 레이어는 아직 학습되지 않았으므로, 이제 당신의 데이터로 파인튜닝해야 합니다.
"""
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


모델과 토크나이저 로드 완료.

주의: 'Some weights of BertForTokenClassification were not initialized...' 메시지는 정상입니다.
이는 사전 학습된 BERT 모델에 토큰 분류를 위한 새로운 레이어(classifier.bias, classifier.weight)가 추가되었기 때문입니다.
이 레이어는 아직 학습되지 않았으므로, 이제 당신의 데이터로 파인튜닝해야 합니다.



### 4.2 성능 지표(Metrics) 정의

모델 학습 중 또는 학습 완료 후 모델의 성능을 평가할 지표를 정의합니다. NER에서는 주로 **정확도(accuracy), 정밀도(precision), 재현율(recall), F1-점수(F1-score)**를 사용합니다. Hugging Face에서 제공하는 `seqeval` 라이브러리를 사용합니다.


In [ ]:
# 평가지표 로드 (Hugging Face에서 제공하는 seqeval 사용)
# load_metric is deprecated, use evaluate.load instead
metric = evaluate.load("seqeval")


def compute_metrics(p):
    predictions, labels = p
    print(f"DEBUG - Inside compute_metrics:")
    print(f"DEBUG - predictions shape before argmax: {p[0].shape}") # 로짓 형태 확인
    print(f"DEBUG - labels shape: {labels.shape}")
    print(f"DEBUG - Sample predictions (after argmax, before filtering): {predictions[0][:10]}") # 첫 번째 샘플 10개
    print(f"DEBUG - Sample labels (before filtering): {labels[0][:10]}") # 첫 번째 샘플 10개

    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    print(f"DEBUG - Sample true_predictions (decoded & filtered): {true_predictions[0][:10]}") # 첫 번째 샘플 10개
    print(f"DEBUG - Sample true_labels (decoded & filtered): {true_labels[0][:10]}") # 첫 번째 샘플 10개
    print(f"DEBUG - Length of true_predictions for first sample: {len(true_predictions[0])}")
    print(f"DEBUG - Length of true_labels for first sample: {len(true_labels[0])}")

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
print("성능 지표 계산 함수 정의 완료.")

성능 지표 계산 함수 정의 완료.


### 4.3 학습 인자(Training Arguments) 및 트레이너(Trainer) 설정

모델을 어떻게 학습시킬지에 대한 설정(하이퍼파라미터)과, Hugging Face `Trainer` 객체를 생성합니다.

**⚠️ 오류 해결 (이전 대화에서 발생했던 문제):**

- `evaluation_strategy`가 `eval_strategy`로 이름이 변경되었습니다. 최신 버전에 맞춰 수정합니다.
- `model_dir`이 정의되지 않았다는 오류는 `0.3 프로젝트 디렉토리 구조 설정` 셀을 다시 실행하면 해결됩니다.


In [ ]:
# 학습 인자 설정
# output_dir: 학습된 모델과 로그가 저장될 경로
# eval_strategy: 'epoch'로 설정하여 각 에포크 종료 시마다 평가 수행
# learning_rate: 학습률 (초기값 2e-5가 일반적입니다.)
# per_device_train_batch_size: GPU당 학습 배치 크기 (GPU 메모리에 따라 조절 가능)
# per_device_eval_batch_size: GPU당 평가 배치 크기
# num_train_epochs: 전체 학습 에포크 수 (데이터셋을 몇 번 반복 학습할지, 데이터가 적으면 과적합 주의)
# weight_decay: 가중치 감쇠 (과적합 방지 기법)
# push_to_hub: 학습된 모델을 Hugging Face Hub에 업로드할지 여부 (지금은 False)
# logging_dir: 학습 로그 저장 경로
# logging_steps: 몇 스텝마다 로그를 출력할지 (데이터셋이 작으면 로그가 자주 안 나올 수 있음)
training_args = TrainingArguments(
    output_dir=model_dir,  # 모델 저장 경로 (0.3단계에서 설정한 model_dir)
    eval_strategy="epoch",  # <-- 'evaluation_strategy'가 'eval_strategy'로 변경됨!
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # GPU 메모리가 부족하면 8, 4 등으로 줄여보세요.
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # 초반에는 적은 에포크로 시작하고, 데이터가 많아지면 늘려볼 수 있습니다.
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir=os.path.join(model_dir, "logs"),
    logging_steps=10,
    report_to="none",  # Colab 환경에서 불필요한 경고 방지 (wandb 등 설정 시 변경)
)

# Trainer 객체 생성
# Trainer는 모델, 학습 인자, 데이터셋, 토크나이저, 성능 지표 계산 함수를 인자로 받아 학습을 관리합니다.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # 학습 데이터셋
    eval_dataset=eval_dataset,  # 평가 데이터셋
    tokenizer=tokenizer,  # 토크나이저 (FutureWarning이 발생할 수 있지만, 현재는 정상 작동)
    compute_metrics=compute_metrics,  # 성능 지표 계산 함수
)

print("학습 인자 및 Trainer 설정 완료.")

/tmp/ipython-input-17-1835957651.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


학습 인자 및 Trainer 설정 완료.


### 4.4 모델 학습 시작

이제 `trainer.train()` 명령으로 모델 학습을 시작합니다. 학습 과정과 평가 결과는 로그로 출력됩니다.


In [ ]:
# 모델 학습 시작
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.441800,0.698927,0.004018,0.014831,0.006323,0.810809
2,0.477300,0.352521,0.078003,0.105932,0.089847,0.893012
3,0.401500,0.301150,0.149425,0.220339,0.178082,0.913735


DEBUG - Inside compute_metrics:
DEBUG - predictions shape before argmax: (61, 512, 11)
DEBUG - labels shape: (61, 512)
DEBUG - Sample predictions (after argmax, before filtering): [[-1.6906476  -0.7724846  -1.3249437  -0.97374505  3.9370596  -0.9235799
  -1.877875   -1.0367949   5.165018   -1.889963    0.66116613]
 [-1.1834552   0.24396372 -0.44478303 -0.22351092  2.2792947  -0.28841445
  -1.845184   -1.0976721   2.519553   -1.3015867   0.505287  ]
 [-1.2466389  -0.84090436  0.5590392  -0.44638476  2.4124904  -0.69581074
  -0.5518167  -0.46675137  2.875707    0.01870449  0.83710414]
 [-1.1758887  -0.4593723  -0.6190254  -0.797266    3.6642334  -0.36649266
  -1.7383738  -0.61791545  4.1529036  -1.6773968   1.107772  ]
 [-1.2912579  -0.70954174 -0.6525979  -0.8274466   3.965962   -1.4350171
  -1.7284806  -1.2037082   3.855284   -1.6505436   0.7188103 ]
 [-1.8264217  -1.0621891  -1.0227355  -0.84778905  3.8427436  -0.66191834
  -1.6566682  -1.14563     3.8590002  -1.1545323   1.1744362 ]


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


DEBUG - Inside compute_metrics:
DEBUG - predictions shape before argmax: (61, 512, 11)
DEBUG - labels shape: (61, 512)
DEBUG - Sample predictions (after argmax, before filtering): [[-2.106447   -1.2209442  -1.0621375  -1.0691075   1.8796865  -0.6413936
  -1.3801854  -0.87555265  5.793545   -1.3072627   0.521834  ]
 [-1.0472993   0.99144024 -0.37101555 -0.2636279   1.1241384   0.11555295
  -1.559196   -0.69929415  0.93298125 -0.6214062   0.35896212]
 [-0.59414023 -0.4419181   0.98723674 -0.49040037  1.4236727  -0.29704487
   0.33308914 -0.28135145  0.59760755  0.25990146  0.6109547 ]
 [-1.2508651  -0.4524014  -0.81024355 -0.60887325  4.70552    -0.70616364
  -1.5417368  -0.04396829  2.3865118  -1.4450315   0.8363676 ]
 [-1.3461545  -0.82766056 -0.7881407  -0.6416364   5.3100214  -1.579544
  -1.6720034  -0.7595059   2.2263007  -1.6750739   0.5125997 ]
 [-1.9405245  -1.0149571  -1.4699165  -0.7125231   5.241595   -0.7462998
  -1.7463461  -0.6409789   2.1123264  -1.2688627   1.1438344 ]
 [

TrainOutput(global_step=48, training_loss=0.6899176587661108, metrics={'train_runtime': 85.9778, 'train_samples_per_second': 8.409, 'train_steps_per_second': 0.558, 'total_flos': 188932927048704.0, 'train_loss': 0.6899176587661108, 'epoch': 3.0})

---


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=512 # Doccano에서 설정한 max_length와 동일하게 설정
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        # --- 디버깅용 print 문 추가 ---
        print(f"\n--- 샘플 {i} ---")
        print(f"원본 words: {examples['tokens'][i]}")
        print(f"원본 ner_tags (word-level): {label}")
        print(f"토큰화된 input_ids (일부): {tokenized_inputs['input_ids'][i][:10]}...")
        print(f"word_ids: {word_ids}")
        # --- 디버깅용 print 문 끝 ---

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # 특수 토큰 (-100)
            if word_idx is None:
                label_ids.append(-100)
            # 첫 번째 서브워드 토큰 (단어의 시작)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # 후속 서브워드 토큰 (단어의 나머지 부분)
            else:
                # B- 태그를 I- 태그로 변경
                # 원래 라벨이 B- 태그인 경우 해당 I- 태그 ID로 변경합니다.
                # 예를 들어, B-QUESTION_ID (ID 1) -> I-QUESTION_ID (ID 2)
                # 이 부분이 중요합니다: id_to_label과 label_to_id를 정확히 사용해야 합니다.
                current_label = id_to_label[label[word_idx]]
                if current_label.startswith("B-"):
                    # 'B-'를 'I-'로 바꾸고 해당 ID를 찾습니다.
                    # 이 로직이 정확한지 확인해야 합니다.
                    # 예: B-QUESTION_ID -> I-QUESTION_ID (ID 1 -> ID 2)
                    i_tag_label = "I-" + current_label[2:]
                    if i_tag_label in label_to_id: # label_to_id 딕셔너리가 필요합니다.
                        label_ids.append(label_to_id[i_tag_label])
                    else:
                        # 해당 I- 태그가 없는 경우 O 태그로 처리하거나, 그대로 B- 태그 유지 (고려 필요)
                        label_ids.append(label[word_idx]) # 혹은 0으로? -> 오류의 원인일 수 있음
                else:
                    label_ids.append(label[word_idx]) # B- 태그가 아닌 경우 그대로 유지
            previous_word_idx = word_idx
        labels.append(label_ids)

        # --- 디버깅용 print 문 추가 ---
        # print(f"aligned_labels (token-level): {label_ids}")
        # print(f"디코딩된 aligned_labels (일부): {[id_to_label[lid] for lid in label_ids if lid != -100][:20]}")
        # --- 디버깅용 print 문 끝 ---

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

## 5. 학습된 모델 평가 및 추론 (테스트)

학습이 완료된 모델을 저장하고, 새로운 텍스트에 대해 개체명 인식을 수행하는 방법을 알아봅니다. 현재 학습 데이터셋의 양이 매우 적으므로, 초기 예측 결과는 만족스럽지 않을 수 있습니다.


### 5.1 학습된 모델 저장

학습이 완료된 모델의 가중치와 설정 파일이 `model_dir` 경로에 저장됩니다. 이렇게 저장된 모델은 나중에 다시 로드하여 사용할 수 있습니다.


In [ ]:
# 학습된 모델 저장
trainer.save_model(model_dir)  # model_dir은 0.3단계에서 설정했던 모델 저장 경로입니다.

print(f"학습된 모델이 '{model_dir}'에 저장되었습니다.")

학습된 모델이 '/content/gdrive/MyDrive/Colab Notebooks/deep-learning-ner-advanced/model'에 저장되었습니다.


### 5.2 저장된 모델 및 토크나이저 로드 (추론 준비)

저장된 모델을 로드하여 새로운 텍스트에 대한 예측(추론)을 수행할 준비를 합니다. 모델을 CPU/GPU에 로드하고 평가 모드로 설정합니다.


In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# 저장된 모델과 토크나이저 로드
loaded_tokenizer = AutoTokenizer.from_pretrained(model_dir)
loaded_model = AutoModelForTokenClassification.from_pretrained(model_dir)

# 모델을 평가 모드로 설정 (드롭아웃 등을 비활성화하여 일관된 예측을 보장)
loaded_model.eval()

# GPU가 있다면 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

print(f"모델과 토크나이저가 '{model_dir}'에서 로드되었습니다. 현재 device: {device}")

모델과 토크나이저가 '/content/gdrive/MyDrive/Colab Notebooks/deep-learning-ner-advanced/model'에서 로드되었습니다. 현재 device: cuda


### 5.3 단일 텍스트에 대한 개체명 인식 추론 함수

임의의 텍스트를 입력받아 모델이 개체명을 예측하고, BIO 태그를 사람이 읽기 쉬운 '개체명 스팬' 형태로 변환하여 출력하는 함수를 정의합니다.


In [ ]:
def predict_ner(text, tokenizer, model, id_to_label, device):
    # 텍스트 토큰화 (모델 입력에 맞게 패딩, 잘림 적용)
    tokenized_input = tokenizer(
        text,
        return_tensors="pt",  # PyTorch 텐서 반환
        truncation=True,
        padding="max_length",
        max_length=512,
    ).to(
        device
    )  # 모델이 있는 디바이스(CPU 또는 GPU)로 입력 텐서 이동

    # 모델 예측 수행
    with torch.no_grad():  # 추론 시에는 기울기 계산 비활성화 (메모리 절약, 속도 향상)
        output = model(**tokenized_input)

    # 예측된 로짓(logits)에서 가장 높은 확률을 가진 레이블 ID 추출
    # squeeze()는 배치 차원(크기 1)을 제거하고, cpu().numpy()로 넘파이 배열로 변환
    predictions = torch.argmax(output.logits, dim=2).squeeze().cpu().numpy()

    # 토큰 및 레이블 디코딩
    tokens = tokenizer.convert_ids_to_tokens(
        tokenized_input["input_ids"].squeeze().cpu().numpy()
    )
    predicted_labels = [id_to_label[p_id] for p_id in predictions]

    # 특수 토큰 및 패딩 토큰 제거 (실제 단어에 대한 예측만 보기 위함)
    # 또한 B-와 I- 태그를 결합하여 개체명 스팬을 출력합니다.

    results = []
    current_entity = ""
    current_label = ""

    # 예측된 토큰과 레이블을 순회하며 개체명 추출
    for token, label in zip(tokens, predicted_labels):
        if token.startswith("##"):  # WordPiece 토크나이저의 서브워드 접두사 제거
            token = token[2:]

        # CLS, SEP, PAD 토큰 등 특수 토큰 제외
        if token in tokenizer.all_special_tokens:  # CLS, SEP, PAD 토큰은 건너뜁니다.
            if current_entity:  # 이전에 수집 중이던 개체명이 있다면 마무리
                results.append((current_entity.strip(), current_label))
                current_entity = ""
                current_label = ""
            continue

        if label.startswith("B-"):  # 새로운 개체명의 시작 (Beginning)
            if current_entity:  # 이전에 수집 중이던 개체명이 있다면 먼저 결과에 추가
                results.append((current_entity.strip(), current_label))
            current_entity = token  # 새로운 개체명 시작
            current_label = label[2:]  # 'B-' 접두사 제거하여 순수 레이블 이름 저장
        elif label.startswith("I-") and current_label and label[2:] == current_label:
            # 현재 토큰이 이전 개체명의 연속 (Inside)이고, 레이블 유형이 일치할 경우
            current_entity += token  # 현재 토큰을 기존 개체명에 추가
        else:  # 'O' 태그이거나, 'I-' 태그인데 이전 레이블과 일치하지 않는 경우
            if current_entity:  # 이전에 수집 중이던 개체명이 있다면 마무리
                results.append((current_entity.strip(), current_label))
            current_entity = ""  # 현재 개체명 초기화
            current_label = ""  # 현재 레이블 초기화

    # 반복문 종료 후 마지막에 남아있는 개체명이 있다면 추가
    if current_entity:
        results.append((current_entity.strip(), current_label))

    return results


print("개체명 인식 추론 함수 정의 완료.")

개체명 인식 추론 함수 정의 완료.


### 5.4 모델 테스트 (예시 문장)

정의한 추론 함수를 사용하여 학습된 모델이 새로운 문장에서 개체명을 얼마나 잘 예측하는지 확인합니다.


In [ ]:
# 테스트할 예시 문장
test_text_1 = """
질의 5 "첨부5"과 같이 감지기 배선을 시공하여도 문제가 없는지 여부. 회신 5 「자동화재탐지설비 및 시각경보기의 화재안전기준(NFSC 203)」 제11조제2호 나목에 따라 감지기 상호간 또는 감지기로부터 수신기에 이르는 감지기 회로의 배선은 「옥내소화전설비의 화재안전기준(NFSC 102)」 [별표 1]에 따른 내화배선 또는 내열배선을 사용하도록 정하고 있습니다. 귀하께서 ”첨부파일5“로 문의하신 감지기 배선 시공방법은 HFIX 전선을 금속제 가요전선관에 수납하여 시공한 부분은 내열 또는 내화배선 시공법에 적합하나, 단열재에서 천장면 난연CD전전관에 수납한 시공방법은 화재안전 기준에서 규정하고 있는 내열ㆍ내화배선에 해당하지 않습니다. 따라서 답변1을 참고하여 내화배선에 따른 시공을 하거나 내열배선에 따른시공으로 배선처리를 하여야 할 것으로 판단됩니다.

"""

# 추론 함수 실행
predicted_entities_1 = predict_ner(
    test_text_1, loaded_tokenizer, loaded_model, id_to_label, device
)

print(f"\n입력 텍스트: {test_text_1}")
print(f"예측된 개체명: {predicted_entities_1}")

# 다른 예시


입력 텍스트: 
질의 5 "첨부5"과 같이 감지기 배선을 시공하여도 문제가 없는지 여부. 회신 5 「자동화재탐지설비 및 시각경보기의 화재안전기준(NFSC 203)」 제11조제2호 나목에 따라 감지기 상호간 또는 감지기로부터 수신기에 이르는 감지기 회로의 배선은 「옥내소화전설비의 화재안전기준(NFSC 102)」 [별표 1]에 따른 내화배선 또는 내열배선을 사용하도록 정하고 있습니다. 귀하께서 ”첨부파일5“로 문의하신 감지기 배선 시공방법은 HFIX 전선을 금속제 가요전선관에 수납하여 시공한 부분은 내열 또는 내화배선 시공법에 적합하나, 단열재에서 천장면 난연CD전전관에 수납한 시공방법은 화재안전 기준에서 규정하고 있는 내열ㆍ내화배선에 해당하지 않습니다. 따라서 답변1을 참고하여 내화배선에 따른 시공을 하거나 내열배선에 따른시공으로 배선처리를 하여야 할 것으로 판단됩니다. 


예측된 개체명: [('질의', 'QUESTION_ID'), ('회신', 'QUESTION_ID')]


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 한국어 -> 영어 번역 모델 로드 (이전과 동일)
ko_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ko-en")
ko_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ko-en").to(device)

# 영어 -> 한국어 번역 모델 로드 (모델 ID 수정)
# 'Helsinki-NLP/opus-mt-en-ko' 대신 'Helsinki-NLP/opus-mt-tc-big-en-ko' 사용
en_ko_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")
en_ko_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko").to(device)

print("번역 모델 로딩 완료.")

Using device: cpu


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/418M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

번역 모델 로딩 완료.


In [17]:
import os
import json
import pandas as pd
from datasets import Dataset, Features, Value, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm.auto import tqdm

# --- 1. Doccano에서 정의한 레이블 목록 (B-, I- 없이) ---
doccano_raw_labels = [
    "QUESTION_ID",
    "QUESTION_CONTENT",
    "ANSWER_ID",
    "ANSWER_CONTENT",
    "LAW_CONTENT",
]

# --- 2. 모델 학습을 위한 최종 BIO 레이블 목록 및 매핑 ---
label_list = ["O"]
for label in doccano_raw_labels:
    label_list.append(f"B-{label}")
    label_list.append(f"I-{label}")

label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}
num_labels = len(label_list)

print(f"모델 학습을 위한 최종 레이블 목록: {label_list}")
print(f"총 레이블 개수: {num_labels}")
print(f"label_to_id: {label_to_id}")
print(f"id_to_label: {id_to_label}")

# --- 3. Doccano에서 내보낸 JSONL 파일 경로 설정 ---
project_root = "/content/gdrive/MyDrive/Colab Notebooks/deep-learning-ner-advanced/"
data_dir = os.path.join(project_root, "data")
labeled_data_file_path = os.path.join(data_dir, "after_datalabeling.jsonl")

# --- 4. 사용할 토크나이저 및 모델 로드 ---
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 번역 모델 로드 (이전 단계에서 수정한 모델 ID 사용)
ko_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ko-en")
ko_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ko-en").to(device)
en_ko_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko") # 수정된 모델 ID
en_ko_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko").to(device)
print("번역 모델 로딩 완료.")


# --- 5. 백 트랜슬레이션 함수 (이전과 동일) ---
def back_translate(text, ko_en_model, ko_en_tokenizer, en_ko_model, en_ko_tokenizer, device):
    # 한국어 -> 영어
    tokenized_text = ko_en_tokenizer.prepare_seq2seq_batch([text], return_tensors='pt').to(device)
    translated_tokens = ko_en_model.generate(**tokenized_text)
    english_text = ko_en_tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    # 영어 -> 한국어
    tokenized_text = en_ko_tokenizer.prepare_seq2seq_batch([english_text], return_tensors='pt').to(device)
    translated_tokens = en_ko_model.generate(**tokenized_text)
    korean_text_augmented = en_ko_tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    return korean_text_augmented

# --- 6. NER 데이터를 증강하는 메인 함수 (수정됨: 라벨 이름을 ID로 변환) ---
def augment_ner_example(example, ko_en_model, ko_en_tokenizer, en_ko_model, en_ko_tokenizer, device):
    text = example['text']
    # Ensure labels are sorted by start index
    labels = sorted(example.get('labels', []), key=lambda x: x[0])

    augmented_text_parts = []
    # new_labels will store [start, end, LABEL_ID]
    new_labels = []
    last_idx = 0
    current_offset = 0

    for start, end, label_name in labels:
        context_part = text[last_idx:start]
        if context_part:
            augmented_context = back_translate(context_part, ko_en_model, ko_en_tokenizer, en_ko_model, en_ko_tokenizer, device)
            augmented_text_parts.append(augmented_context)
            current_offset += len(augmented_context)

        entity_part = text[start:end]
        augmented_text_parts.append(entity_part)

        new_start = current_offset
        new_end = current_offset + len(entity_part)

        # Convert label_name string to its corresponding ID
        # Need to handle cases where the label_name might not be in label_to_id (shouldn't happen with valid Doccano export)
        label_id = label_to_id.get(f"B-{label_name}", 0) # Use B- tag ID for span start, default to 0 ('O') if not found

        new_labels.append([new_start, new_end, label_id]) # Store as [start, end, LABEL_ID]

        current_offset += len(entity_part)
        last_idx = end

    final_context_part = text[last_idx:]
    if final_context_part:
        augmented_context = back_translate(final_context_part, ko_en_model, ko_en_tokenizer, en_ko_model, en_ko_tokenizer, device)
        augmented_text_parts.append(augmented_context)

    new_text = "".join(augmented_text_parts)
    return {"text": new_text, "labels": new_labels} # 'labels' now stores [start, end, LABEL_ID]


# --- 7. Doccano JSONL 로드 및 데이터 증강 ---
original_data_raw = []
try:
    with open(labeled_data_file_path, "r", encoding="utf-8") as f:
        for line in f:
            original_data_raw.append(json.loads(line))
except FileNotFoundError:
    print(f"오류: 레이블링된 파일 '{labeled_data_file_path}'을(를) 찾을 수 없습니다.")
    # Exit gracefully or handle this case appropriately if not running in interactive mode
    # In Colab, just printing the error is often sufficient for interactive debugging
    pass


# Convert original data labels from strings to IDs immediately
original_data_processed = []
for example in original_data_raw:
    processed_labels = []
    # Ensure labels exist and are sorted
    labels = sorted(example.get('labels', []), key=lambda x: x[0])
    for start, end, label_name in labels:
         # Convert label_name string to its corresponding ID (using B- tag ID for span start)
        label_id = label_to_id.get(f"B-{label_name}", 0) # Default to 0 ('O') if not found
        processed_labels.append([start, end, label_id]) # Store as [start, end, LABEL_ID]
    original_data_processed.append({"text": example['text'], "labels": processed_labels})


N_augment = 5 # Number of samples to augment

augmented_examples = []
if original_data_processed: # Only augment if original data was loaded successfully
    print(f"총 {len(original_data_processed)}개의 샘플 중 {N_augment}개에 대해 데이터 증강을 시작합니다...")

    # Data augmentation execution (returns text and [start, end, LABEL_ID] labels)
    for i in tqdm(range(min(N_augment, len(original_data_processed)))):
        augmented_example = augment_ner_example(original_data_processed[i], ko_en_model, ko_en_tokenizer, en_ko_model, en_ko_tokenizer, device)
        augmented_examples.append(augmented_example)

    print("\n--- 데이터 증강 예시 ---")
    for i in range(min(5, len(augmented_examples))): # Print up to 5 augmented samples
        print(f"\n[샘플 {i+1}]")
        print("원본 텍스트:", original_data_processed[i]['text'])
        print("증강 텍스트:", augmented_examples[i]['text'])
        # Decode label IDs back to names for printing clarity
        decoded_original_labels = [[s, e, id_to_label[l_id]] for s, e, l_id in original_data_processed[i]['labels']]
        decoded_augmented_labels = [[s, e, id_to_label[l_id]] for s, e, l_id in augmented_examples[i]['labels']]
        print("원본 라벨 (디코딩):", decoded_original_labels)
        print("증강 라벨 (디코딩):", decoded_augmented_labels)
else:
    print("원본 데이터를 로드하지 못하여 데이터 증강을 건너뜁니다.")


# --- 8. 증강된 데이터와 원본 데이터를 합치고 Hugging Face Dataset 형식으로 최종 변환 ---

# Combine original (processed with IDs) and augmented data
combined_raw_data_processed = original_data_processed + augmented_examples

# Step 8.1: Create initial Hugging Face Dataset from the processed list
# 'labels' column now contains lists of [start, end, LABEL_ID]
# Define the features here to ensure 'labels' is treated correctly as a sequence of integer IDs.
# We need a structure that can handle lists of lists of integers.
# Define the features for the dataset before converting from list
dataset_features = Features({
    'text': Value('string'),
    'labels': Sequence(Sequence(Value('int32'))) # Use int32 for label IDs
})


# Create the dataset using the defined features
# Dataset.from_list can infer features, but explicitly defining helps avoid errors
# and ensures the structure for 'labels' is correctly interpreted.
try:
    raw_hf_dataset = Dataset.from_list(combined_raw_data_processed, features=dataset_features)
    print(f"\n임시 Hugging Face Dataset으로 변환된 샘플 수 (토큰화 전): {len(raw_hf_dataset)}")
    print(raw_hf_dataset[0]) # Check the structure and data types
except Exception as e:
    print(f"오류: 임시 Hugging Face Dataset 생성 중 오류 발생: {e}")
    # Exit or handle error
    pass


# Step 8.2: Tokenization and label alignment function (Dataset.map() will use this)
def tokenize_and_align_labels_for_dataset(examples):
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512, # Use the same max_length as defined earlier
        padding="max_length",
        return_offsets_mapping=True
    )

    labels = []
    # examples["text"] is a list, examples["labels"] is a list of lists of [start, end, LABEL_ID] lists
    for batch_idx, (text, ner_tags_char_offsets_with_ids) in enumerate(zip(examples["text"], examples["labels"])):
        # ner_tags_char_offsets_with_ids is [[start, end, LABEL_ID], ...]
        word_ids = tokenized_inputs.word_ids(batch_index=batch_idx)
        offset_mapping = tokenized_inputs["offset_mapping"][batch_idx]

        token_labels_ids = [-100] * len(word_ids) # Initialize token-level label ID list

        # Assign BIO labels for each token
        # ner_tags_char_offsets_with_ids is assumed to be sorted by start index
        for token_idx, current_word_idx in enumerate(word_ids):
            if current_word_idx is None: # Special tokens (CLS, SEP, PAD)
                token_labels_ids[token_idx] = -100
            else: # Regular word tokens
                token_start_offset, token_end_offset = offset_mapping[token_idx]
                current_token_label_id = 0 # Default label is 'O' (ID 0)

                # Check which annotation (entity) the current token falls into
                for ann_start, ann_end, ann_label_id in ner_tags_char_offsets_with_ids:
                    # If the token's offset is fully within the annotation's range
                    if ann_start <= token_start_offset and token_end_offset <= ann_end:
                         # If the token's start offset is the same as the annotation's start offset, it's the Beginning token (B-)
                        if ann_start == token_start_offset:
                             # Get the B- tag ID for this label_id
                            # Check if the B- tag exists for this ID (should always if ID is valid)
                            b_tag_label = id_to_label[ann_label_id] # Get label name from ID
                            if b_tag_label.startswith("B-"): # Ensure it's already a B- tag (from our processing)
                                current_token_label_id = ann_label_id # Keep the B- tag ID
                            else: # Should not happen if previous step is correct, default to O
                                current_token_label_id = 0
                        else: # Otherwise, it's an Inside token (I-)
                            # Get the I- tag ID for this label_id
                            # The original label_id stored in ner_tags_char_offsets_with_ids was the B- tag ID.
                            # We need to find the corresponding I- tag ID.
                            b_tag_label_name = id_to_label[ann_label_id][2:] # Get the pure label name (e.g., 'QUESTION_ID')
                            i_tag_label = f"I-{b_tag_label_name}" # Construct the I- tag name (e.g., 'I-QUESTION_ID')
                            current_token_label_id = label_to_id.get(i_tag_label, 0) # Get the I- tag ID, default to O if not found
                        break # Found the annotation, no need to search further

                token_labels_ids[token_idx] = current_token_label_id
        labels.append(token_labels_ids)

    tokenized_inputs["labels"] = labels
    # offset_mapping is not needed for model training, remove it
    tokenized_inputs.pop("offset_mapping")
    return tokenized_inputs


# Step 8.3: Apply map() function for tokenization and label alignment
# batch=True for efficiency
if 'raw_hf_dataset' in locals() and raw_hf_dataset is not None:
    try:
        final_hf_dataset = raw_hf_dataset.map(
            tokenize_and_align_labels_for_dataset,
            batched=True,
            remove_columns=raw_hf_dataset.column_names # Remove original 'text', 'labels' columns
        )

        # Step 8.4: Set format for the final Dataset
        final_hf_dataset.set_format(
            type="torch",
            columns=['input_ids', 'attention_mask', 'labels']
        )

        print(f"\n최종 Hugging Face Dataset (토큰화 및 라벨 정렬 완료) 샘플 수: {len(final_hf_dataset)}")
        print("\n최종 변환된 Hugging Face Dataset 첫 번째 샘플:")
        print(final_hf_dataset[0])

        # Final train/test split (same as before)
        train_test_split_dataset = final_hf_dataset.train_test_split(test_size=0.2, seed=42)
        train_dataset = train_test_split_dataset["train"]
        eval_dataset = train_test_split_dataset["test"]

        print(f"\n학습 데이터셋 샘플 수: {len(train_dataset)}")
        print(f"평가 데이터셋 샘플 수: {len(eval_dataset)}")

    except Exception as e:
        print(f"오류: 최종 Hugging Face Dataset 처리 중 오류 발생: {e}")

else:
    print("임시 Hugging Face Dataset이 생성되지 않아 최종 처리를 건너뜁니다.")

모델 학습을 위한 최종 레이블 목록: ['O', 'B-QUESTION_ID', 'I-QUESTION_ID', 'B-QUESTION_CONTENT', 'I-QUESTION_CONTENT', 'B-ANSWER_ID', 'I-ANSWER_ID', 'B-ANSWER_CONTENT', 'I-ANSWER_CONTENT', 'B-LAW_CONTENT', 'I-LAW_CONTENT']
총 레이블 개수: 11
label_to_id: {'O': 0, 'B-QUESTION_ID': 1, 'I-QUESTION_ID': 2, 'B-QUESTION_CONTENT': 3, 'I-QUESTION_CONTENT': 4, 'B-ANSWER_ID': 5, 'I-ANSWER_ID': 6, 'B-ANSWER_CONTENT': 7, 'I-ANSWER_CONTENT': 8, 'B-LAW_CONTENT': 9, 'I-LAW_CONTENT': 10}
id_to_label: {0: 'O', 1: 'B-QUESTION_ID', 2: 'I-QUESTION_ID', 3: 'B-QUESTION_CONTENT', 4: 'I-QUESTION_CONTENT', 5: 'B-ANSWER_ID', 6: 'I-ANSWER_ID', 7: 'B-ANSWER_CONTENT', 8: 'I-ANSWER_CONTENT', 9: 'B-LAW_CONTENT', 10: 'I-LAW_CONTENT'}


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


번역 모델 로딩 완료.
총 302개의 샘플 중 5개에 대해 데이터 증강을 시작합니다...


  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:4073: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)



--- 데이터 증강 예시 ---

[샘플 1]
원본 텍스트: 질의 1 연면적 450㎡인 특정소방대상물에 최초 건축물 사용승인시에 비상경보설비 설치가 되지 않은 경우 건축허가일과 사용승인일 중 소방시설설치기준 적용일은? 회신 1 건축물 등의 신축ㆍ증축ㆍ개축ㆍ재축ㆍ이전ㆍ용도변경 또는 대수선의 허가 ㆍ협의 및 사용승인의 권한이 있는 행정기관은 소방시설법 제7조제1항에 따라 소재지를 관할하는 소방본부장이나 소방서장의 동의를 받아야 하므로, 건축허가등과 관련한 협의과정이 누락되었다면, 건축허가 신청일을 기준으로 소방시설의 설치기준을 적용합니다. 
증강 텍스트: 질의 1엑셀 호텔링이나 CD연면적 450㎡인 특정소방대상물에 최초 건축물 사용승인시에 비상경보설비 설치가 되지 않은 경우 건축허가일과 사용승인일 중 소방시설설치기준 적용일은?엑셀 호텔링이나 CD회신 1엑셀 호텔링이나 CD건축물 등의 신축ㆍ증축ㆍ개축ㆍ재축ㆍ이전ㆍ용도변경 또는 대수선의 허가 ㆍ협의 및 사용승인의 권한이 있는 행정기관은 소방시설법 제7조제1항에 따라 소재지를 관할하는 소방본부장이나 소방서장의 동의를 받아야 하므로, 건축허가등과 관련한 협의과정이 누락되었다면, 건축허가 신청일을 기준으로 소방시설의 설치기준을 적용합니다.엑셀 호텔링이나 CD
원본 라벨 (디코딩): [[0, 4, 'B-QUESTION_ID'], [5, 87, 'B-QUESTION_CONTENT'], [88, 92, 'B-ANSWER_ID'], [93, 156, 'B-ANSWER_CONTENT'], [156, 168, 'B-LAW_CONTENT'], [168, 268, 'B-ANSWER_CONTENT']]
증강 라벨 (디코딩): [[0, 4, 'O'], [15, 97, 'O'], [108, 112, 'O'], [123, 186, 'O'], [186, 198, 'O'], [198, 298, 'O']]

[샘플 2]
원본 텍스트: 질의 2 최초 사업허가승인월이 ‘13년 6월인 대상물의 사업이 변경되어 최종 사업허가승인월이 19년 2월인 경우, 

Map:   0%|          | 0/307 [00:00<?, ? examples/s]


최종 Hugging Face Dataset (토큰화 및 라벨 정렬 완료) 샘플 수: 307

최종 변환된 Hugging Face Dataset 첫 번째 샘플:
{'labels': tensor([-100,    1,    2,    3,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    5,    6,    7,    8,    8,    8,    8,    8,    8,
           8,    8,    8,    8,    8,    8,    8,    8,    8,    8,    8,    8,
           8,    8,    8,    8,    8,    8,    8,    8,    8,    8,    8,    8,
           8,    8,    8,    8,    8,    8,    8,    8,    8,    8,    9,   10,
          10,   10,   10,   10,   10,   10,   10,    7,    8,    8,    8,    8,
           8,    8,    8,    8,    8,    8,    8,    8,    8,    8,    8,    8,
           8,    8,    8,    8,    8,    8,    8,    8,    8,    8,    8,    8,
           8,    8,

### 5.5 예측 결과 분석 및 현재 모델의 한계

1. NER 모델의 성능향상 기법을 사용하지 않았을 때

    예측 결과는 정확도가 매우 낮거나, 기대했던 개체명을 제대로 인식하지 못했습니다. (예: 대부분의 단어를 `QUESTION_CONTENT`로 분류하거나, 주요 개체명을 놓치는 등).

    **이것은 지극히 정상적인 현상입니다.** 현재 모델의 성능이 낮은 주된 이유는 다음과 같습니다:

    - **매우 적은 학습 데이터:** 우리가 사용한 학습 데이터셋은 15개(약 5%)의 샘플에 불과합니다. BERT와 같은 딥러닝 모델은 수많은 패턴을 학습해야 하므로, 단 12개의 샘플로는 의미 있는 특징을 학습하고 일반화하기에 턱없이 부족합니다.
    - **'O' 태그 편향:** 모델은 학습 데이터에서 'O'(Other) 태그가 훨씬 많기 때문에, 안전하게 대부분의 토큰을 'O'로 예측하거나, 특정 라벨로 뭉뚱그려 예측하려는 경향이 있습니다.    
      
   => **Weakly Supervised Learning** 을 적용하기로 결정.



2. NER 모델에 Weakly Supervised Learning 을 적용했을 때

    질의-회신 세트에서 '질의 1', '회신 1' 과 같은 QUESTION_ID 및 ANSWER_ID 를 정확하게 예측했습니다. 하지면 여전히 그 외의 개체명을 인식하지 못하고 있습니다.
    
    =>  **Back-Translation** 기법을 활용한 **Data Augmentation** 을 적용하기로 결정.

3. NER 모델에 Active Learning 을 적용했을 때


    
---


## 6. 성능 향상을 위한 추가 단계 (결론)

지금까지 개체명 인식 모델 학습을 위한 전체 파이프라인을 성공적으로 구축했습니다. 이제 구축된 파이프라인을 통해 모델의 성능을 향상시키는 데 집중할 차례입니다.

### 6.1 더 많은 데이터 라벨링 (가장 중요!)

모델 성능 향상의 90% 이상은 **데이터의 양과 질**에 달려 있습니다. 현재 가장 필요한 것은 Doccano로 돌아가서 **더 많은 질의/회신 문서를 라벨링하는 것**입니다.

- **목표:** 최소 수백 개에서 수천 개 이상의 질의/회신 쌍을 라벨링하는 것을 목표로 합니다.
- **다양성 확보:** 다양한 문맥과 표현을 가진 텍스트에 대해 라벨링하여 모델이 일반화된 패턴을 학습할 수 있도록 도와야 합니다. (예: '소방시설법령'이 다양한 문장에서 어떻게 등장하는지 등)
- **일관성 유지:** 라벨링 규칙을 명확히 하고 일관되게 적용하는 것이 중요합니다.

**워크플로우 반복:**

1.  Doccano에서 추가 데이터 라벨링
2.  `Export only approved documents`를 체크하여 `.jsonl` 파일 내보내기
3.  Google Drive에 업로드
4.  **Colab 노트북의 `3. Colab에서 라벨링된 데이터 로드 및 전처리` 셀부터 다시 실행하여 모델 재학습**

이 과정을 반복하면서 모델의 Precision, Recall, F1-score가 점진적으로 향상되는 것을 확인할 수 있을 것입니다.

### 6.2 하이퍼파라미터 튜닝

충분한 데이터가 확보된 후에는, `4.3 학습 인자(Training Arguments) 및 트레이너(Trainer) 설정` 섹션의 하이퍼파라미터(예: `learning_rate`, `per_device_train_batch_size`, `num_train_epochs`)를 조정하여 모델 성능을 최적화할 수 있습니다.

### 6.3 모델 아키텍처 및 전이 학습 고려

현재 `klue/bert-base`는 좋은 시작점입니다. 하지만 더 전문적인 도메인(예: 법률)에 특화된 사전 학습 모델이 있다면 이를 활용하는 것도 고려해볼 수 있습니다. (한국어 법률 특화 모델은 제한적일 수 있습니다.)

---


## 7. 프로젝트 회고 및 배운 점

이번 프로젝트를 통해 딥러닝 모델을 실제 업무에 적용하는 전체 과정을 경험하며 많은 것을 배우고 느꼈습니다. 단순히 이론으로만 알던 개념들을 직접 부딪히고 해결하며 얻은 교훈들을 정리했습니다.

### 기술적 성장 및 경험

- **클라우드 기반 GPU 활용의 필요성:** 딥러닝 모델 학습처럼 대규모 연산이 필요한 작업은 일반 로컬 PC 환경에서 수행하기 어렵다는 것을 체감했습니다. **Google Colab**이 제공하는 무료 **T4 GPU**는 이러한 제약을 극복하고, 비용 효율적으로 모델을 학습하고 실험할 수 있는 훌륭한 대안이었습니다.

- **Docker와 컨테이너 환경 경험:** 데이터 라벨링 도구인 **Doccano**를 설치하고 실행하기 위해 **Docker**를 처음 사용해보았습니다. 이를 통해 애플리케이션을 격리된 환경에서 손쉽게 배포하고 실행하는 컨테이너 기술의 강력함을 이해하게 되었고, 복잡한 설치 과정 없이 필요한 도구를 빠르게 구축하는 경험을 쌓을 수 있었습니다.

- **엔드-투-엔드(End-to-End) 파이프라인 구축:** 데이터 전처리부터 라벨링, 모델 학습, 평가, 그리고 추론에 이르기까지의 전체 머신러닝 파이프라인을 직접 설계하고 구축했습니다. 각 단계가 어떻게 유기적으로 연결되는지, 그리고 각 단계에서 어떤 점을 고려해야 하는지에 대한 실질적인 이해를 높일 수 있었습니다.

### 딥러닝 모델과 데이터에 대한 깊은 이해

- **'Garbage In, Garbage Out'의 실감:** 모델의 성능은 결국 데이터의 양과 질에 의해 결정된다는 것을 뼈저리게 느꼈습니다. 특히, **라벨링된 데이터 샘플의 수가 많을수록 모델의 정확도가 비례하여 향상**되는 것을 직접 확인했습니다. 초기 단계에서 적은 수의 샘플로 학습했을 때 모델이 거의 작동하지 않았던 경험은 양질의 데이터 확보가 얼마나 중요한지 깨닫게 해주었습니다.

- **사전 학습 모델(Pre-trained Model)의 위력:** `klue/bert-base`와 같은 사전 학습된 모델을 기반으로 파인튜닝하는 것이 왜 효율적인지를 이해했습니다. 밑바닥부터 모든 것을 학습시키는 대신, 이미 방대한 한국어 데이터를 학습한 모델을 활용함으로써 비교적 적은 데이터로도 특정 도메인의 작업을 수행할 수 있다는 전이 학습의 개념을 실제로 적용해볼 수 있었습니다.

### 현실적인 한계와 성과

- **고품질 학습 데이터셋 구축의 어려움:** 지도 학습(Supervised Learning) 기반의 NER 모델을 훈련시키기 위해 1만 2천 개의 모든 데이터를 라벨링할 필요는 없습니다. 핵심은 모델이 전체 데이터의 패턴을 학습할 수 있을 만큼, **충분한 양의 대표적인 샘플들을 고품질로 라벨링**하는 것입니다. 일단 모델이 잘 학습되면, 나머지 라벨링되지 않은 데이터는 모델이 자동으로 처리해줄 수 있습니다. 하지만, 원하는 성능을 내기 위한 '충분한 양의 샘플'을 만드는 것 자체가 혼자서는 매우 힘든 작업이었습니다. 이로 인해 100% 완벽한 정확도를 가진 모델을 만들지는 못했지만, 지도 학습의 핵심 원리와 데이터의 중요성을 체감하는 계기가 되었습니다.

- **그럼에도 불구하고, 성공적인 자동화:** 비록 모델이 완벽하지는 않았지만, 이 프로젝트를 통해 개발한 자동화된 파싱 시스템은 기존의 **수작업으로 질의-회신 쌍을 검증하던 방식에 비해 업무 효율을 압도적으로 향상**시켰습니다. 반복적인 작업을 자동화함으로써 시간을 절약하고, 더 중요한 분석 작업에 집중할 수 있게 되었다는 점에서 이 프로젝트는 매우 성공적이었습니다. 이는 '완벽함'을 추구하기보다 '개선'을 목표로 하는 것의 중요성을 일깨워 주었습니다.
